<a href="https://colab.research.google.com/github/NayeonKimdev/FFmpeg/blob/main/FFmpeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================================================================
# 1단계: 환경 설정 및 샘플 비디오 준비
# =============================================================================

# FFmpeg 설치 및 확인
!apt update -qq && apt install -y ffmpeg
!ffmpeg -version | head -5

# 필요한 라이브러리 설치
!pip install -q moviepy opencv-python pillow numpy matplotlib pandas

import subprocess
import os
import time
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Video, HTML
import json

# 작업 디렉토리 생성
os.makedirs('ffmpeg_toolkit', exist_ok=True)
os.makedirs('ffmpeg_toolkit/input', exist_ok=True)
os.makedirs('ffmpeg_toolkit/output', exist_ok=True)
os.chdir('ffmpeg_toolkit')

print("✅ 환경 설정 완료!")

37 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libf

In [2]:
# =============================================================================
# 2단계: 샘플 비디오 생성 (실제 비디오가 없을 경우)
# =============================================================================

def create_sample_video():
    """테스트용 샘플 비디오 생성"""
    cmd = """
    ffmpeg -f lavfi -i testsrc2=duration=10:size=1920x1080:rate=30 \
    -f lavfi -i sine=frequency=1000:duration=10 \
    -c:v libx264 -pix_fmt yuv420p -c:a aac \
    input/sample_video.mp4 -y
    """

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ 샘플 비디오 생성 완료: input/sample_video.mp4")
        return "input/sample_video.mp4"
    else:
        print("❌ 샘플 비디오 생성 실패:", result.stderr)
        return None

# 샘플 비디오 생성
sample_video = create_sample_video()

# 비디오 정보 확인
def get_video_info(video_path):
    """비디오 정보 추출"""
    cmd = f'ffprobe -v quiet -print_format json -show_format -show_streams "{video_path}"'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode == 0:
        info = json.loads(result.stdout)
        return info
    else:
        print(f"❌ 비디오 정보 추출 실패: {result.stderr}")
        return None

# 원본 비디오 정보 출력
if sample_video:
    info = get_video_info(sample_video)
    if info:
        video_stream = next((s for s in info['streams'] if s['codec_type'] == 'video'), None)
        audio_stream = next((s for s in info['streams'] if s['codec_type'] == 'audio'), None)

        print(f"\n📹 원본 비디오 정보:")
        print(f"파일 크기: {os.path.getsize(sample_video) / (1024*1024):.2f} MB")
        if video_stream:
            print(f"해상도: {video_stream['width']}x{video_stream['height']}")
            print(f"비디오 코덱: {video_stream['codec_name']}")
            print(f"프레임레이트: {eval(video_stream['r_frame_rate']):.2f} fps")
        if audio_stream:
            print(f"오디오 코덱: {audio_stream['codec_name']}")

✅ 샘플 비디오 생성 완료: input/sample_video.mp4

📹 원본 비디오 정보:
파일 크기: 7.39 MB
해상도: 1920x1080
비디오 코덱: h264
프레임레이트: 30.00 fps
오디오 코덱: aac


In [ ]:
# =============================================================================
# 3단계: 비디오 압축 실습
# =============================================================================

class VideoCompressor:
    def __init__(self):
        self.compression_profiles = {
            'high_quality': {
                'name': 'High Quality (CRF 18)',
                'params': '-c:v libx264 -crf 18 -preset medium -c:a aac -b:a 192k'
            },
            'balanced': {
                'name': 'Balanced (CRF 23)',
                'params': '-c:v libx264 -crf 23 -preset medium -c:a aac -b:a 128k'
            },
            'small_size': {
                'name': 'Small Size (CRF 28)',
                'params': '-c:v libx264 -crf 28 -preset slow -c:a aac -b:a 96k'
            },
            'mobile': {
                'name': 'Mobile Optimized',
                'params': '-c:v libx264 -crf 26 -preset fast -vf scale=1280:720 -c:a aac -b:a 96k'
            },
            'web_streaming': {
                'name': 'Web Streaming (VP9)',
                'params': '-c:v libvpx-vp9 -crf 30 -b:v 0 -c:a libopus -b:a 128k'
            }
        }

    def compress_video(self, input_path, profile_name, output_suffix=""):
        """지정된 프로파일로 비디오 압축"""
        if profile_name not in self.compression_profiles:
            print(f"❌ 알 수 없는 프로파일: {profile_name}")
            return None

        profile = self.compression_profiles[profile_name]

        # 출력 파일명 생성
        input_name = Path(input_path).stem
        if profile_name == 'web_streaming':
            output_path = f"output/{input_name}_{profile_name}{output_suffix}.webm"
        else:
            output_path = f"output/{input_name}_{profile_name}{output_suffix}.mp4"

        # FFmpeg 명령어 생성
        cmd = f'ffmpeg -i "{input_path}" {profile["params"]} "{output_path}" -y'

        print(f"🔄 압축 시작: {profile['name']}")
        print(f"명령어: {cmd}")

        start_time = time.time()
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        end_time = time.time()

        if result.returncode == 0:
            processing_time = end_time - start_time
            file_size = os.path.getsize(output_path) / (1024*1024)

            print(f"✅ 압축 완료!")
            print(f"   파일: {output_path}")
            print(f"   크기: {file_size:.2f} MB")
            print(f"   처리시간: {processing_time:.1f}초")

            return {
                'profile': profile_name,
                'output_path': output_path,
                'file_size_mb': file_size,
                'processing_time': processing_time,
                'success': True
            }
        else:
            print(f"❌ 압축 실패: {result.stderr}")
            return {
                'profile': profile_name,
                'success': False,
                'error': result.stderr
            }

# 압축 실습 실행
compressor = VideoCompressor()
compression_results = []

if sample_video:
    print("🎬 다양한 압축 프로파일로 비디오 압축 시작...\n")

    for profile_name in compressor.compression_profiles:
        result = compressor.compress_video(sample_video, profile_name)
        if result:
            compression_results.append(result)
        print("-" * 50)

In [4]:
# =============================================================================
# 4단계: 포맷 변환 실습
# =============================================================================

class FormatConverter:
    def __init__(self):
        self.conversion_profiles = {
            'mp4_h264': {
                'name': 'MP4 (H.264)',
                'extension': 'mp4',
                'params': '-c:v libx264 -c:a aac'
            },
            'webm_vp9': {
                'name': 'WebM (VP9)',
                'extension': 'webm',
                'params': '-c:v libvpx-vp9 -c:a libopus'
            },
            'mov_prores': {
                'name': 'MOV (ProRes - if available)',
                'extension': 'mov',
                'params': '-c:v libx264 -c:a aac'  # ProRes는 라이센스 문제로 H.264 대체
            },
            'gif_animated': {
                'name': 'Animated GIF',
                'extension': 'gif',
                'params': '-vf "fps=10,scale=480:-1:flags=lanczos,palettegen=stats_mode=diff" -t 3'
            }
        }

    def convert_format(self, input_path, target_format):
        """포맷 변환 실행"""
        if target_format not in self.conversion_profiles:
            print(f"❌ 지원하지 않는 포맷: {target_format}")
            return None

        profile = self.conversion_profiles[target_format]
        input_name = Path(input_path).stem
        output_path = f"output/{input_name}_converted.{profile['extension']}"

        # GIF는 특별 처리 (팔레트 생성 후 변환)
        if target_format == 'gif_animated':
            # 1단계: 팔레트 생성
            palette_cmd = f'ffmpeg -i "{input_path}" -vf "fps=10,scale=480:-1:flags=lanczos,palettegen=stats_mode=diff" -t 3 palette.png -y'
            subprocess.run(palette_cmd, shell=True, capture_output=True)

            # 2단계: GIF 생성
            cmd = f'ffmpeg -i "{input_path}" -i palette.png -lavfi "fps=10,scale=480:-1:flags=lanczos,paletteuse=dither=bayer:bayer_scale=5" -t 3 "{output_path}" -y'
        else:
            cmd = f'ffmpeg -i "{input_path}" {profile["params"]} "{output_path}" -y'

        print(f"🔄 변환 시작: {profile['name']}")

        start_time = time.time()
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        end_time = time.time()

        if result.returncode == 0:
            processing_time = end_time - start_time
            file_size = os.path.getsize(output_path) / (1024*1024)

            print(f"✅ 변환 완료!")
            print(f"   파일: {output_path}")
            print(f"   크기: {file_size:.2f} MB")
            print(f"   처리시간: {processing_time:.1f}초")

            return {
                'format': target_format,
                'output_path': output_path,
                'file_size_mb': file_size,
                'processing_time': processing_time,
                'success': True
            }
        else:
            print(f"❌ 변환 실패: {result.stderr}")
            return None

# 포맷 변환 실습 실행
converter = FormatConverter()
conversion_results = []

if sample_video:
    print("\n🔄 포맷 변환 실습 시작...\n")

    for format_name in ['mp4_h264', 'webm_vp9', 'gif_animated']:
        result = converter.convert_format(sample_video, format_name)
        if result:
            conversion_results.append(result)
        print("-" * 50)


🔄 포맷 변환 실습 시작...

🔄 변환 시작: MP4 (H.264)
✅ 변환 완료!
   파일: output/sample_video_converted.mp4
   크기: 7.02 MB
   처리시간: 20.1초
--------------------------------------------------
🔄 변환 시작: WebM (VP9)
✅ 변환 완료!
   파일: output/sample_video_converted.webm
   크기: 7.88 MB
   처리시간: 178.5초
--------------------------------------------------
🔄 변환 시작: Animated GIF
✅ 변환 완료!
   파일: output/sample_video_converted.gif
   크기: 0.35 MB
   처리시간: 1.0초
--------------------------------------------------


In [5]:
# =============================================================================
# 5단계: 고급 필터 체인 실습
# =============================================================================

class AdvancedFilters:
    def __init__(self):
        self.filter_presets = {
            'color_correction': {
                'name': 'Color Correction',
                'filter': 'eq=brightness=0.1:contrast=1.2:saturation=1.1'
            },
            'sharpen': {
                'name': 'Sharpen Filter',
                'filter': 'unsharp=5:5:1.0:5:5:0.0'
            },
            'vintage_look': {
                'name': 'Vintage Look',
                'filter': 'curves=vintage,vignette=PI/4'
            },
            'speed_up_2x': {
                'name': 'Speed Up 2x',
                'filter': 'setpts=0.5*PTS',
                'audio_filter': 'atempo=2.0'
            },
            'slow_motion': {
                'name': 'Slow Motion (0.5x)',
                'filter': 'setpts=2.0*PTS',
                'audio_filter': 'atempo=0.5'
            }
        }

    def apply_filter(self, input_path, filter_name):
        """필터 적용"""
        if filter_name not in self.filter_presets:
            print(f"❌ 알 수 없는 필터: {filter_name}")
            return None

        preset = self.filter_presets[filter_name]
        input_name = Path(input_path).stem
        output_path = f"output/{input_name}_{filter_name}.mp4"

        # 기본 필터 명령어
        cmd_parts = [f'ffmpeg -i "{input_path}"']

        # 비디오 필터 추가
        if 'filter' in preset:
            cmd_parts.append(f'-vf "{preset["filter"]}"')

        # 오디오 필터 추가 (속도 변경 시)
        if 'audio_filter' in preset:
            cmd_parts.append(f'-af "{preset["audio_filter"]}"')

        # 출력 설정
        cmd_parts.extend(['-c:v libx264', '-c:a aac', f'"{output_path}"', '-y'])

        cmd = ' '.join(cmd_parts)

        print(f"🎨 필터 적용: {preset['name']}")
        print(f"명령어: {cmd}")

        start_time = time.time()
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        end_time = time.time()

        if result.returncode == 0:
            processing_time = end_time - start_time
            file_size = os.path.getsize(output_path) / (1024*1024)

            print(f"✅ 필터 적용 완료!")
            print(f"   파일: {output_path}")
            print(f"   크기: {file_size:.2f} MB")
            print(f"   처리시간: {processing_time:.1f}초")

            return {
                'filter': filter_name,
                'output_path': output_path,
                'file_size_mb': file_size,
                'processing_time': processing_time,
                'success': True
            }
        else:
            print(f"❌ 필터 적용 실패: {result.stderr}")
            return None

# 필터 실습 실행
filter_processor = AdvancedFilters()
filter_results = []

if sample_video:
    print("\n🎨 고급 필터 체인 실습 시작...\n")

    for filter_name in ['color_correction', 'speed_up_2x', 'vintage_look']:
        result = filter_processor.apply_filter(sample_video, filter_name)
        if result:
            filter_results.append(result)
        print("-" * 50)


🎨 고급 필터 체인 실습 시작...

🎨 필터 적용: Color Correction
명령어: ffmpeg -i "input/sample_video.mp4" -vf "eq=brightness=0.1:contrast=1.2:saturation=1.1" -c:v libx264 -c:a aac "output/sample_video_color_correction.mp4" -y
✅ 필터 적용 완료!
   파일: output/sample_video_color_correction.mp4
   크기: 7.59 MB
   처리시간: 21.9초
--------------------------------------------------
🎨 필터 적용: Speed Up 2x
명령어: ffmpeg -i "input/sample_video.mp4" -vf "setpts=0.5*PTS" -af "atempo=2.0" -c:v libx264 -c:a aac "output/sample_video_speed_up_2x.mp4" -y
✅ 필터 적용 완료!
   파일: output/sample_video_speed_up_2x.mp4
   크기: 3.77 MB
   처리시간: 12.4초
--------------------------------------------------
🎨 필터 적용: Vintage Look
명령어: ffmpeg -i "input/sample_video.mp4" -vf "curves=vintage,vignette=PI/4" -c:v libx264 -c:a aac "output/sample_video_vintage_look.mp4" -y
✅ 필터 적용 완료!
   파일: output/sample_video_vintage_look.mp4
   크기: 4.84 MB
   처리시간: 42.8초
--------------------------------------------------


In [7]:
# =============================================================================
# 6단계: 결과 분석 및 시각화
# =============================================================================

def analyze_results():
    """처리 결과 분석 및 시각화"""

    # 전역 변수 확인
    global compression_results, conversion_results, filter_results

    # 변수가 정의되지 않은 경우 빈 리스트로 초기화
    if 'compression_results' not in globals():
        compression_results = []
    if 'conversion_results' not in globals():
        conversion_results = []
    if 'filter_results' not in globals():
        filter_results = []

    print("📊 FFmpeg 실습 결과 분석")
    print("=" * 50)

    # 1. 압축 결과 분석
    successful_compressions = [r for r in compression_results if r.get('success', False)]
    if successful_compressions:
        print("\n🗜️ 압축 결과:")

        df_compression = pd.DataFrame(successful_compressions)
        display_cols = ['profile_name', 'file_size_mb', 'processing_time']

        # 컬럼이 존재하는지 확인 후 출력
        available_cols = [col for col in display_cols if col in df_compression.columns]
        if available_cols:
            print(df_compression[available_cols].to_string(index=False))

        # 시각화
        if len(successful_compressions) >= 2:
            try:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

                # 파일 크기 비교
                profiles = [r.get('profile', r.get('profile_name', f"Profile {i}")) for i, r in enumerate(successful_compressions)]
                sizes = [r['file_size_mb'] for r in successful_compressions]
                times = [r['processing_time'] for r in successful_compressions]

                ax1.bar(profiles, sizes, color='skyblue', alpha=0.7)
                ax1.set_title('파일 크기 비교', fontsize=12)
                ax1.set_ylabel('크기 (MB)')
                ax1.tick_params(axis='x', rotation=45)

                ax2.bar(profiles, times, color='lightcoral', alpha=0.7)
                ax2.set_title('처리 시간 비교', fontsize=12)
                ax2.set_ylabel('시간 (초)')
                ax2.tick_params(axis='x', rotation=45)

                plt.tight_layout()
                plt.savefig('output/compression_analysis.png', dpi=150, bbox_inches='tight')
                plt.show()

                print("📈 차트가 생성되었습니다: output/compression_analysis.png")

            except Exception as e:
                print(f"⚠️ 차트 생성 중 오류: {e}")
    else:
        print("❌ 압축 결과가 없습니다.")

    # 2. 변환 결과 분석
    if conversion_results:
        print(f"\n🔄 변환 결과: {len(conversion_results)}개 완료")
        for result in conversion_results:
            name = result.get('format_name', result.get('format', 'Unknown'))
            size = result.get('file_size_mb', 0)
            time_taken = result.get('processing_time', 0)
            print(f"   • {name}: {size:.2f} MB ({time_taken:.1f}초)")

    # 3. 필터 결과 분석
    if filter_results:
        print(f"\n🎨 필터 결과: {len(filter_results)}개 완료")
        for result in filter_results:
            name = result.get('filter_name', result.get('filter', 'Unknown'))
            size = result.get('file_size_mb', 0)
            time_taken = result.get('processing_time', 0)
            print(f"   • {name}: {size:.2f} MB ({time_taken:.1f}초)")

    # 4. 생성된 파일 목록
    output_files = []
    if os.path.exists('output'):
        output_files = [f for f in os.listdir('output') if f.endswith(('.mp4', '.webm', '.gif', '.png'))]

    print(f"\n📁 생성된 파일 ({len(output_files)}개):")
    total_size = 0
    for file in sorted(output_files):
        file_path = f"output/{file}"
        if os.path.exists(file_path):
            size_mb = os.path.getsize(file_path) / (1024*1024)
            total_size += size_mb
            print(f"   📄 {file} ({size_mb:.2f} MB)")

    print(f"\n💾 총 생성 용량: {total_size:.2f} MB")

    # 5. 요약 통계
    print(f"\n📋 실습 요약:")
    print(f"   ✅ 압축: {len(successful_compressions)}개")
    print(f"   ✅ 변환: {len(conversion_results)}개")
    print(f"   ✅ 필터: {len(filter_results)}개")
    print(f"   📁 총 파일: {len(output_files)}개")

    return {
        'compressions': len(successful_compressions),
        'conversions': len(conversion_results),
        'filters': len(filter_results),
        'total_files': len(output_files),
        'total_size_mb': total_size
    }

# 결과 분석 실행
results_summary = analyze_results()

📊 FFmpeg 실습 결과 분석
❌ 압축 결과가 없습니다.

🔄 변환 결과: 3개 완료
   • mp4_h264: 7.02 MB (20.1초)
   • webm_vp9: 7.88 MB (178.5초)
   • gif_animated: 0.35 MB (1.0초)

🎨 필터 결과: 3개 완료
   • color_correction: 7.59 MB (21.9초)
   • speed_up_2x: 3.77 MB (12.4초)
   • vintage_look: 4.84 MB (42.8초)

📁 생성된 파일 (6개):
   📄 sample_video_color_correction.mp4 (7.59 MB)
   📄 sample_video_converted.gif (0.35 MB)
   📄 sample_video_converted.mp4 (7.02 MB)
   📄 sample_video_converted.webm (7.88 MB)
   📄 sample_video_speed_up_2x.mp4 (3.77 MB)
   📄 sample_video_vintage_look.mp4 (4.84 MB)

💾 총 생성 용량: 31.45 MB

📋 실습 요약:
   ✅ 압축: 0개
   ✅ 변환: 3개
   ✅ 필터: 3개
   📁 총 파일: 6개


In [9]:
# =============================================================================
# 7단계: 실무형 배치 처리 스크립트
# =============================================================================

import subprocess
import os
import time
from pathlib import Path
import json

class BatchProcessor:
    """배치 처리용 클래스 (실무에서 활용) - 독립적으로 작동"""

    def __init__(self):
        self.processing_log = []

        # 압축 프로파일 내장
        self.compression_profiles = {
            'high_quality': {
                'name': 'High Quality (CRF 18)',
                'params': '-c:v libx264 -crf 18 -preset medium -c:a aac -b:a 192k'
            },
            'balanced': {
                'name': 'Balanced (CRF 23)',
                'params': '-c:v libx264 -crf 23 -preset medium -c:a aac -b:a 128k'
            },
            'small_size': {
                'name': 'Small Size (CRF 28)',
                'params': '-c:v libx264 -crf 28 -preset slow -c:a aac -b:a 96k'
            },
            'mobile': {
                'name': 'Mobile Optimized',
                'params': '-c:v libx264 -crf 26 -preset fast -vf scale=1280:720 -c:a aac -b:a 96k'
            }
        }

        # 변환 프로파일 내장
        self.conversion_profiles = {
            'webm_vp9': {
                'name': 'WebM (VP9)',
                'extension': 'webm',
                'params': '-c:v libvpx-vp9 -crf 30 -b:v 0 -c:a libopus -b:a 128k'
            },
            'mp4_h264': {
                'name': 'MP4 (H.264)',
                'extension': 'mp4',
                'params': '-c:v libx264 -c:a aac'
            }
        }

        # 필터 프로파일 내장
        self.filter_profiles = {
            'color_correction': {
                'name': 'Color Correction',
                'params': '-vf "eq=brightness=0.1:contrast=1.2:saturation=1.1" -c:v libx264 -c:a aac'
            },
            'speed_up_2x': {
                'name': 'Speed Up 2x',
                'params': '-vf "setpts=0.5*PTS" -af "atempo=2.0" -c:v libx264 -c:a aac'
            },
            'sharpen': {
                'name': 'Sharpen Filter',
                'params': '-vf "unsharp=5:5:1.0:5:5:0.0" -c:v libx264 -c:a aac'
            }
        }

    def compress_video_internal(self, input_path, profile, output_suffix=""):
        """내장 압축 함수"""
        if profile not in self.compression_profiles:
            print(f"❌ 알 수 없는 압축 프로파일: {profile}")
            return None

        profile_config = self.compression_profiles[profile]
        input_name = Path(input_path).stem
        output_path = f"output/{input_name}_{profile}{output_suffix}.mp4"

        cmd = f'ffmpeg -i "{input_path}" {profile_config["params"]} "{output_path}" -y'

        print(f"🔄 압축: {profile_config['name']}")

        start_time = time.time()
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        end_time = time.time()

        if result.returncode == 0:
            processing_time = end_time - start_time
            file_size = os.path.getsize(output_path) / (1024*1024)

            print(f"✅ 완료: {file_size:.2f} MB ({processing_time:.1f}초)")

            return {
                'operation': 'compress',
                'profile': profile,
                'profile_name': profile_config['name'],
                'output_path': output_path,
                'file_size_mb': file_size,
                'processing_time': processing_time,
                'success': True
            }
        else:
            print(f"❌ 압축 실패: {result.stderr}")
            return {'success': False, 'error': result.stderr}

    def convert_video_internal(self, input_path, target_format, output_suffix=""):
        """내장 변환 함수"""
        if target_format not in self.conversion_profiles:
            print(f"❌ 알 수 없는 변환 포맷: {target_format}")
            return None

        profile = self.conversion_profiles[target_format]
        input_name = Path(input_path).stem
        output_path = f"output/{input_name}_{target_format}{output_suffix}.{profile['extension']}"

        cmd = f'ffmpeg -i "{input_path}" {profile["params"]} "{output_path}" -y'

        print(f"🔄 변환: {profile['name']}")

        start_time = time.time()
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        end_time = time.time()

        if result.returncode == 0:
            processing_time = end_time - start_time
            file_size = os.path.getsize(output_path) / (1024*1024)

            print(f"✅ 완료: {file_size:.2f} MB ({processing_time:.1f}초)")

            return {
                'operation': 'convert',
                'format': target_format,
                'format_name': profile['name'],
                'output_path': output_path,
                'file_size_mb': file_size,
                'processing_time': processing_time,
                'success': True
            }
        else:
            print(f"❌ 변환 실패: {result.stderr}")
            return {'success': False, 'error': result.stderr}

    def filter_video_internal(self, input_path, filter_name, output_suffix=""):
        """내장 필터 함수"""
        if filter_name not in self.filter_profiles:
            print(f"❌ 알 수 없는 필터: {filter_name}")
            return None

        profile = self.filter_profiles[filter_name]
        input_name = Path(input_path).stem
        output_path = f"output/{input_name}_{filter_name}{output_suffix}.mp4"

        cmd = f'ffmpeg -i "{input_path}" {profile["params"]} "{output_path}" -y'

        print(f"🎨 필터: {profile['name']}")

        start_time = time.time()
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        end_time = time.time()

        if result.returncode == 0:
            processing_time = end_time - start_time
            file_size = os.path.getsize(output_path) / (1024*1024)

            print(f"✅ 완료: {file_size:.2f} MB ({processing_time:.1f}초)")

            return {
                'operation': 'filter',
                'filter': filter_name,
                'filter_name': profile['name'],
                'output_path': output_path,
                'file_size_mb': file_size,
                'processing_time': processing_time,
                'success': True
            }
        else:
            print(f"❌ 필터 실패: {result.stderr}")
            return {'success': False, 'error': result.stderr}

    def process_directory(self, input_dir, output_dir, operation='compress', profile='balanced'):
        """디렉토리 내 모든 비디오 파일을 배치 처리"""

        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.webm']
        input_files = []

        # 비디오 파일 찾기
        for ext in video_extensions:
            input_files.extend(Path(input_dir).glob(f"*{ext}"))

        if not input_files:
            print(f"❌ {input_dir}에서 비디오 파일을 찾을 수 없습니다.")
            return

        print(f"🎬 배치 처리 시작: {len(input_files)}개 파일")
        print(f"📋 작업: {operation} (프로파일: {profile})")

        os.makedirs(output_dir, exist_ok=True)

        # 각 파일 처리
        for i, input_file in enumerate(input_files, 1):
            print(f"\n[{i}/{len(input_files)}] 처리 중: {input_file.name}")

            try:
                # 작업 타입별 처리
                if operation == 'compress':
                    result = self.compress_video_internal(str(input_file), profile, "_batch")
                elif operation == 'convert':
                    result = self.convert_video_internal(str(input_file), profile, "_batch")
                elif operation == 'filter':
                    result = self.filter_video_internal(str(input_file), profile, "_batch")
                else:
                    print(f"❌ 알 수 없는 작업: {operation}")
                    continue

                # 성공한 경우 로그에 추가
                if result and result['success']:
                    # 배치 처리 결과를 지정된 output_dir로 이동
                    src_path = result['output_path']
                    dst_path = os.path.join(output_dir, os.path.basename(src_path))

                    if src_path != dst_path and os.path.exists(src_path):
                        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
                        os.rename(src_path, dst_path)
                        result['output_path'] = dst_path
                        print(f"📁 이동 완료: {dst_path}")

                    self.processing_log.append(result)

            except Exception as e:
                print(f"❌ 처리 중 오류 발생: {e}")
                continue

        print(f"\n✅ 배치 처리 완료!")
        self.print_batch_summary()

    def print_batch_summary(self):
        """배치 처리 요약 출력"""
        if not self.processing_log:
            print("❌ 처리된 파일이 없습니다.")
            return

        total_files = len(self.processing_log)
        total_size = sum(r['file_size_mb'] for r in self.processing_log)
        total_time = sum(r['processing_time'] for r in self.processing_log)

        print(f"\n📊 배치 처리 요약:")
        print(f"   ✅ 처리된 파일: {total_files}개")
        print(f"   💾 총 용량: {total_size:.2f} MB")
        print(f"   ⏱️ 총 처리시간: {total_time:.1f}초")
        print(f"   📈 평균 처리시간: {total_time/total_files:.1f}초/파일")

        # 작업별 통계
        operations = {}
        for result in self.processing_log:
            op = result.get('operation', 'unknown')
            if op not in operations:
                operations[op] = 0
            operations[op] += 1

        print(f"\n📋 작업별 통계:")
        for op, count in operations.items():
            print(f"   • {op}: {count}개")

    def get_available_profiles(self):
        """사용 가능한 프로파일 목록 반환"""
        print("📋 사용 가능한 프로파일:")
        print("\n🗜️ 압축 프로파일:")
        for profile, config in self.compression_profiles.items():
            print(f"   • {profile}: {config['name']}")

        print("\n🔄 변환 프로파일:")
        for profile, config in self.conversion_profiles.items():
            print(f"   • {profile}: {config['name']}")

        print("\n🎨 필터 프로파일:")
        for profile, config in self.filter_profiles.items():
            print(f"   • {profile}: {config['name']}")

# =============================================================================
# 배치 처리 실행 및 데모
# =============================================================================

# 배치 처리 인스턴스 생성
batch_processor = BatchProcessor()

# 사용 가능한 프로파일 확인
batch_processor.get_available_profiles()

# 현재 input 디렉토리 확인
if os.path.exists('input') and os.listdir('input'):
    print(f"\n📁 input 디렉토리에 {len(os.listdir('input'))}개 파일 발견")

    # 여러 작업 데모
    demos = [
        ('compress', 'balanced', '균형 잡힌 압축'),
        ('compress', 'small_size', '최대 압축'),
        ('convert', 'webm_vp9', 'WebM 변환'),
        ('filter', 'color_correction', '색상 보정')
    ]

    for operation, profile, description in demos:
        print(f"\n{'='*60}")
        print(f"🎬 배치 작업: {description}")
        print(f"{'='*60}")

        # 새로운 배치 프로세서 인스턴스 (로그 분리)
        demo_processor = BatchProcessor()
        output_subdir = f'output/batch_{operation}_{profile}'

        demo_processor.process_directory('input', output_subdir, operation, profile)

        print(f"📁 결과는 {output_subdir}에서 확인하세요.")

else:
    print("❌ input 디렉토리에 비디오 파일이 없습니다.")
    print("💡 샘플 비디오를 생성하려면 이전 셀들을 먼저 실행하세요.")

# =============================================================================
# 고급 배치 처리 기능 데모
# =============================================================================

class AdvancedBatchProcessor(BatchProcessor):
    """고급 배치 처리 기능 추가"""

    def __init__(self):
        super().__init__()
        self.quality_metrics = []

    def analyze_video_quality(self, original_path, processed_path):
        """비디오 품질 분석 (PSNR 계산)"""
        try:
            # PSNR 계산 (Peak Signal-to-Noise Ratio)
            cmd = f'ffmpeg -i "{processed_path}" -i "{original_path}" -lavfi "psnr=stats_file=psnr.log" -f null - 2>/dev/null'
            subprocess.run(cmd, shell=True, capture_output=True)

            # PSNR 결과 파싱 (실제로는 더 복잡한 파싱 필요)
            if os.path.exists('psnr.log'):
                with open('psnr.log', 'r') as f:
                    content = f.read()
                # 여기서는 간단히 파일 크기 비율로 대체
                original_size = os.path.getsize(original_path)
                processed_size = os.path.getsize(processed_path)
                compression_ratio = processed_size / original_size

                os.remove('psnr.log')  # 정리
                return compression_ratio
            else:
                return None
        except Exception as e:
            print(f"⚠️ 품질 분석 오류: {e}")
            return None

    def batch_with_quality_analysis(self, input_dir, output_dir, operation='compress', profile='balanced'):
        """품질 분석이 포함된 배치 처리"""
        print("📊 품질 분석이 포함된 고급 배치 처리 시작")

        # 기본 배치 처리 실행
        self.process_directory(input_dir, output_dir, operation, profile)

        # 품질 분석 수행
        if self.processing_log:
            print("\n🔍 품질 분석 중...")

            video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.webm']
            original_files = []

            for ext in video_extensions:
                original_files.extend(Path(input_dir).glob(f"*{ext}"))

            for original_file in original_files:
                # 처리된 파일 찾기
                processed_files = [r for r in self.processing_log
                                 if Path(r['output_path']).stem.startswith(original_file.stem)]

                for processed_result in processed_files:
                    ratio = self.analyze_video_quality(str(original_file), processed_result['output_path'])
                    if ratio:
                        quality_info = {
                            'original_file': str(original_file),
                            'processed_file': processed_result['output_path'],
                            'compression_ratio': ratio,
                            'size_reduction': (1 - ratio) * 100
                        }
                        self.quality_metrics.append(quality_info)

                        print(f"   📊 {original_file.name} -> 압축률: {ratio:.2f} (크기 {quality_info['size_reduction']:.1f}% 감소)")

# 고급 배치 처리 데모
if os.path.exists('input') and os.listdir('input'):
    print(f"\n{'='*60}")
    print("🚀 고급 배치 처리 데모")
    print(f"{'='*60}")

    advanced_processor = AdvancedBatchProcessor()
    advanced_processor.batch_with_quality_analysis('input', 'output/advanced_batch', 'compress', 'balanced')

print("\n🎉 FFmpeg 배치 처리 실습 완료!")
print("📁 결과물은 output/ 폴더의 각 하위 디렉토리에서 확인하세요.")
print("💼 이 배치 처리 시스템을 GitHub에 업로드하여 포트폴리오로 활용하세요!")

# 최종 결과 요약
print(f"\n{'='*60}")
print("📋 전체 결과 요약")
print(f"{'='*60}")

total_output_files = []
if os.path.exists('output'):
    for root, dirs, files in os.walk('output'):
        for file in files:
            if file.endswith(('.mp4', '.webm', '.gif')):
                total_output_files.append(os.path.join(root, file))

total_size = sum(os.path.getsize(f) / (1024*1024) for f in total_output_files)

print(f"📁 총 생성된 비디오 파일: {len(total_output_files)}개")
print(f"💾 총 용량: {total_size:.2f} MB")
print(f"🎯 포트폴리오 포인트: 자동화된 비디오 처리 파이프라인 구축 완료!")

📋 사용 가능한 프로파일:

🗜️ 압축 프로파일:
   • high_quality: High Quality (CRF 18)
   • balanced: Balanced (CRF 23)
   • small_size: Small Size (CRF 28)
   • mobile: Mobile Optimized

🔄 변환 프로파일:
   • webm_vp9: WebM (VP9)
   • mp4_h264: MP4 (H.264)

🎨 필터 프로파일:
   • color_correction: Color Correction
   • speed_up_2x: Speed Up 2x
   • sharpen: Sharpen Filter

📁 input 디렉토리에 1개 파일 발견

🎬 배치 작업: 균형 잡힌 압축
🎬 배치 처리 시작: 1개 파일
📋 작업: compress (프로파일: balanced)

[1/1] 처리 중: sample_video.mp4
🔄 압축: Balanced (CRF 23)
✅ 완료: 7.01 MB (21.6초)
📁 이동 완료: output/batch_compress_balanced/sample_video_balanced_batch.mp4

✅ 배치 처리 완료!

📊 배치 처리 요약:
   ✅ 처리된 파일: 1개
   💾 총 용량: 7.01 MB
   ⏱️ 총 처리시간: 21.6초
   📈 평균 처리시간: 21.6초/파일

📋 작업별 통계:
   • compress: 1개
📁 결과는 output/batch_compress_balanced에서 확인하세요.

🎬 배치 작업: 최대 압축
🎬 배치 처리 시작: 1개 파일
📋 작업: compress (프로파일: small_size)

[1/1] 처리 중: sample_video.mp4
🔄 압축: Small Size (CRF 28)
✅ 완료: 3.39 MB (27.4초)
📁 이동 완료: output/batch_compress_small_size/sample_video_small_size_batch.mp4

✅ 배치 처리 완료!
